Se carga el dataset que se utilizara

In [ ]:
from pandas import DataFrame
import pandas as pd

dataset =  DataFrame(pd.read_csv('windserie.csv', header=None))
print(dataset.head)

Se aplica media movil en los datos.

In [ ]:
from matplotlib import pyplot

pyplot.style.use("ggplot")
pyplot.plot(dataset[1:150])
pyplot.title('Datos Entrada')
pyplot.figure(num=None, figsize=(8, 6), dpi=320, facecolor='w', edgecolor='k')
pyplot.show()

dataset=dataset.rolling(window=8, min_periods=1).mean()

pyplot.plot(dataset[1:150])
pyplot.title('Media móvil Aplicada')
pyplot.figure(num=None, figsize=(8, 6), dpi=320, facecolor='w', edgecolor='k')
pyplot.show()

Se define el tamaño de ventana que tendra la serie de tiempos, y se transforma a una colección de datos para el entrenamiento supervisado de la red.

In [ ]:
import Tools as tls

size = 5

dataset = tls.series_to_supervised(dataset,size, n_out=1, dropnan = True)
print(dataset)

Se normalizan los datos entre 0 y 1

In [ ]:
from sklearn import preprocessing

datasetnorm = preprocessing.minmax_scale(dataset, feature_range=(0, 1))
print(datasetnorm)
datasetnorm = DataFrame(datasetnorm)
print(datasetnorm.head)

Preparamos los datos para alimentar la red para entrenamiento y prueba.

In [ ]:
import numpy as np
from sklearn.model_selection import train_test_split

zy = datasetnorm.iloc[:, -1:]
zx = datasetnorm.iloc[:, :-1].values
X = np.array(zx, dtype='float64')
y = np.array(zy, dtype='float64')
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=.3, random_state=20)

print(X_train.shape, y_train.shape) 

X_train = X_train.reshape((X_train.shape[0], 1, X_train.shape[1]))
X_test = X_test.reshape((X_test.shape[0], 1, X_test.shape[1]))
y_train = y_train.reshape((y_train.shape[0], 1, y_train.shape[1]))
y_test = y_test.reshape((y_test.shape[0], 1, y_test.shape[1]))

print(X_train.shape, y_train.shape) 

Contruimos nuestro modelo y compilamos

In [ ]:
from keras.models import Sequential
from keras.layers import Dense

model = Sequential() 
model.add(Dense(8, activation = 'relu', input_shape=(X_train.shape[1], X_train.shape[2]))) # importante declarar correctamente shape
model.add(Dense(20, activation='relu'))
model.add(Dense(1))

model.compile(loss = 'mse', optimizer = 'adam', metrics=['mse','mae'])

Entrenamos nuestro modelo y guardamos datos en "history"

In [ ]:
history = model.fit(X_train, y_train, epochs=50, batch_size=24, validation_data=(X_test, y_test), verbose=1, shuffle=False)

Graficamos...

In [ ]:
pyplot.style.use("ggplot")
pyplot.plot(history.history['mean_squared_error'], label='mse')
pyplot.plot(history.history['val_mean_squared_error'], label='val_mse')
pyplot.xlabel('Epochs')
pyplot.ylabel('Value')
pyplot.legend()
pyplot.show()
pyplot.plot(history.history["mean_absolute_error"], label="mae")
pyplot.plot(history.history["val_mean_absolute_error"], label="val_mae")
pyplot.xlabel('Epochs')
pyplot.ylabel('Value')
pyplot.legend()
pyplot.show()

Se realiza una predicción con los datos de prueba, y estos son comparados.

In [ ]:
predictions = model.predict(X_test, 10, verbose=2)
predicted = predictions.transpose(2,0,1).reshape(-1,predictions.shape[1])
y_test = y_test.transpose(2,0,1).reshape(-1,y_test.shape[1])

pyplot.plot(y_test[0:100,], label='Real')
pyplot.plot(predicted[0:100,], label='Predicción')
pyplot.title('Valor predicción contra real')
pyplot.xlabel('Epochs')
pyplot.ylabel('Value')
pyplot.legend()
pyplot.figure(num=None, figsize=(8, 6), dpi=320, facecolor='w', edgecolor='k')
pyplot.show()